# Processing Landsat5/8 timeseries Images for 2000-2020 

### LandSat 5
https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C02_T1_L2

SR_B4 (near infrared), SR_B3 (red)

### LandSat 8
https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA#bands

SR_B5 (Near infrared), SR_B4 (Red)

In [1]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import datetime
from datetime import datetime
import configparser
import os

from time import sleep
from pathlib import Path


ee.Authenticate()
ee.Initialize()


# Read settings

In [2]:
config = configparser.ConfigParser()
config.read("config.ini")

# GEE products
cfg_ls5_product = config["gee"]["landsat5"]
cfg_ls8_product = config["gee"]["landsat8"]
cfg_nuts = config["gee"]["NUTS"]
cfg_NDVI_LS_mean_ts_ASSET = config["gee"]["NDVI_LS_mean_ts_ASSET"] # Landsat full timeseries 
cfg_NDVI_LS_mean_composite_ASSET = config["gee"]["NDVI_LS_mean_composite_ASSET"]  # Landsat composite
cfg_NDVI_LS_EL_mean_percentiles_2_95_mean_CSV = config["gee"]["NDVI_LS_mean_percentiles_2_95_mean_CSV"] # Csv file to save percentiles per NUTS
cfg_GDRIVE_EXPORT_DIRECTORY = config["gee"]["GDRIVE_EXPORT_DIRECTORY"] # Directory in GEE to export results
cfg_FUA = config["gee"]["FUA"]
cfg_gee_assets=config["gee"]["gee_assets"]

# Dates
cfg_ls5_startDate =config["dates"]["ls5_startDate"]
cfg_ls5_endDate = config["dates"]["ls5_endDate"]

cfg_ls8_startDate = config["dates"]["ls8_startDate"]
cfg_ls8_endDate = config["dates"]["ls8_endDate"]


# Settings

cfg_OUTPUT_DIR =  config["settings"]["OUTPUT_DIR"] # Directory in local machine
cfg_TILES =  int(config["settings"]["TILES"])

cfg_NUTS_FILTER_COLUMN = config["settings"]["NUTS_FILTER_COLUMN"]
cfg_NUTS_CNTR_CODE = config["settings"]["NUTS_CNTR_CODE"] 

cfg_SUN_ELEVATION = int(config["settings"]["SUN_ELEVATION"])


OUTPUT_DIR = Path(os.getcwd() ).parent  / cfg_OUTPUT_DIR
TILES=cfg_TILES




# Load Image collections

In [3]:

DATES_LS5=(cfg_ls5_startDate, cfg_ls5_endDate)
DATES_LS8=(cfg_ls8_startDate,cfg_ls8_endDate)


ls5 = ee.ImageCollection(cfg_ls5_product)
ls8 = ee.ImageCollection(cfg_ls8_product)


# Read  and filter NUTS


In [ ]:
# Read NUTS
nuts = ee.FeatureCollection(cfg_nuts)


#Read FUA
fua = ee.FeatureCollection(cfg_FUA)



fua_bb = fua.geometry().bounds()

# Create a bounding box from min/max coordinates

nuts = nuts.filterBounds(fua_bb)


# Process Landsat 5

In [9]:
# Apply filters to the ImageCollection
ls5 = (ls5 
    .filterDate(*DATES_LS5) 
    .filterBounds(nuts.geometry().bounds()))

    
# Filter the ImageCollection to keep only images with SUN_ELEVATION > angle
ls5 = ls5.filter(ee.Filter.gt('SUN_ELEVATION', cfg_SUN_ELEVATION))



In [ ]:
# Applies scaling factors to Landsat 5 imagery.

def apply_scale_factorsL5(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

  
ls5 = ls5.map(apply_scale_factorsL5);


def bitwiseExtract(input, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return input.rightShift(fromBit).bitwiseAnd(mask)


def maskL5sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  FillCloudBitMask = 1 << 0
  DilatedCloudBitMask = 1 << 1
  cloudsBitMask = 1 << 3
  cloudShadowBitMask = 1 << 4
  snowsBitMask = 1 << 5
  WaterBitMask = 1 << 7
  
  # Get the pixel QA band.
  qa = image.select('QA_PIXEL')

  
  mask =   (bitwiseExtract(qa, 8, 9).lte(1) # keep only low confidence or None (1 and 0)
  .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) 
  .And(qa.bitwiseAnd(cloudShadowBitMask).eq(0)) 
  .And(qa.bitwiseAnd(snowsBitMask).eq(0)) 
  .And(qa.bitwiseAnd(FillCloudBitMask).eq(0)) 
  .And(qa.bitwiseAnd(DilatedCloudBitMask).eq(0)) 
  .And(qa.bitwiseAnd(WaterBitMask).eq(0))) 
  
  #  we don't want any channels saturated at all
  # https://gis.stackexchange.com/questions/363929/how-to-apply-a-bitmask-for-radiometric-saturation-qa-in-a-image-collection-eart
  saturationMask = image.select('QA_RADSAT').eq(0);
  
           
  #Return the masked image
  maskedimage = image.updateMask(mask).updateMask(saturationMask)

  maskedimage = maskedimage.addBands(
    ee.Image([qa.bitwiseAnd(WaterBitMask).eq(0).rename('Water')])
    ).copyProperties(image).set('system:time_start', image.get('system:time_start'));
    
    
  return (maskedimage)

# Remove clouds, snow
landsat5FiltMasked = ls5.map(maskL5sr)

# Function to add NDVI, time, and constant variables to Landsat 8 imagery.
def addVariables5(image):
  #Return the image with the added bands.
  return (image
  # Add an NDVI band. And apply coefficient to intercalibrate with Landsat 8
  .addBands(image.normalizedDifference(['SR_B4', 'SR_B3']).multiply(0.9723).add(0.0235).rename('NDVI'))
  .addBands(image.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI')))
  
  
  
imgP5 = landsat5FiltMasked.map(addVariables5)
imgO5 = imgP5.select(['SR_B1', # blue
                      'SR_B2', # green
                      'SR_B3',# red
                      'SR_B4', # IR
                      'NDVI', 
                      'NDWI']).map(lambda x: x.rename(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'NDVI', 'NDWI'])) # rename, identical to Landsat8


####  set values for NDVI in range [0,1]
def set_values(image):
    ndvi = image.select("NDVI")
    ndvi_clamped = ndvi.expression("b(0) < 0 ? 0 : (b(0) > 1 ? 1 : b(0))")
    return image.addBands(ndvi_clamped,  overwrite=True)

imgO5 = imgO5.map(lambda img: set_values(img))



def func_apr5(year):
    """Calculate annual mean for Bands"""
    date_start = ee.Date.fromYMD(year, 1, 1)
    date_end = date_start.advance(1, "year")

    image1 = (imgO5.filterDate(date_start, date_end) 
        .reduce(ee.Reducer.mean(), TILES) 
        .set({"year": year}) 
        .set({"system:time_start": date_start.millis(), "system:time_end": date_end.millis()}))

    return(image1)


# for each year return mean composite of bands
annual_mean_NDVI5 = ee.ImageCollection.fromImages(
    ee.List.sequence(
        ee.Date(DATES_LS5[0]).get("year").getInfo(),
        ee.Date(DATES_LS5[1]).get("year").getInfo() - 1,
    ).map(func_apr5)
)


# LandSat5: 2002 mean composite image  

In [7]:


# Get the 2002 image from the annual_mean_NDVI5 collection
mapYear=2002
first_image = annual_mean_NDVI5.filter(ee.Filter.eq("year", mapYear)).first()

# Define visualization parameters
vis_params = {
    "bands": ["NDVI_mean"],  # Change bands if needed
    "min": -1,
    "max": 1, 
    "palette": ["red", "yellow", "green"]
}

# Create a map
Map = geemap.Map(center=[51.94, 16.35], zoom=5)  # Adjust center and zoom as needed

# Add the first image to the map
Map.addLayer(first_image, vis_params, f"{mapYear} mean_NDVI5 Image")

Map.addLayer(nuts, {'color': 'green'}, "NUTS2", shown=False)
Map.addLayer(nuts.geometry().bounds(), {'fillcolor':'rgba(0, 0, 255, 0)'}, "NUTS2 bounds", shown=False)
Map

Map(center=[51.94, 16.35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
# Export the image of year X
geemap.ee_export_image(
    first_image.select('NDVI_mean'),
    filename=f'/home/leonidas/PARA/1_Projects/Greening_EU_V2/output/NDVI_{mapYear}.tif',
    scale=100,
    crs='EPSG:3035',
    region=nuts,
    file_per_band=False
)

In [ ]:
# Export the image of year X to Google Drive
geemap.ee_export_image_to_drive(
    image=first_image.select('NDVI_mean'),
    description=f'NDVI_{mapYear}.tif',
    folder='EXPORT_GEE',     # Google Drive folder name
    fileNamePrefix=f'NDVI_{mapYear}',    # Output filename
    scale=100,
    crs='EPSG:3035',
    region=nuts.geometry().bounds(),
    fileFormat='GeoTIFF',
    maxPixels=1e13
)


# Process LandSat 8


In [ ]:
ls8 = (ls8
    .filterDate(*DATES_LS8) 
    .filterBounds(nuts.geometry().bounds()))



# Filter the LandSat8 ImageCollection to keep only images with SUN_ELEVATION > 35
ls8 = ls8.filter(ee.Filter.gt('SUN_ELEVATION', cfg_SUN_ELEVATION))

In [9]:
# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
  return (image.addBands(opticalBands, None, True)
              .addBands(thermalBands, None, True))
  
ls8 = ls8.map(applyScaleFactors);

In [10]:


def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  FillCloudBitMask = 1 << 0
  DilatedCloudBitMask = 1 << 1
  cloudsBitMask = 1 << 3
  cloudShadowBitMask = 1 << 4
  snowsBitMask = 1 << 5
  WaterBitMask = 1 << 7
  
  # Get the pixel QA band.
  qa = image.select('QA_PIXEL')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
    .And(bitwiseExtract(qa, 8, 9).lte(1)) \
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
    .And(qa.bitwiseAnd(snowsBitMask).eq(0)) \
    .And(qa.bitwiseAnd(FillCloudBitMask).eq(0)) \
    .And(qa.bitwiseAnd(DilatedCloudBitMask).eq(0)) \
    .And(qa.bitwiseAnd(WaterBitMask).eq(0))
  
  
  #  we don't want any channels saturated at all
  # https://gis.stackexchange.com/questions/363929/how-to-apply-a-bitmask-for-radiometric-saturation-qa-in-a-image-collection-eart
  saturationMask = image.select('QA_RADSAT').eq(0);
  
           
  #Return the masked image
  maskedimage = image.updateMask(mask).updateMask(saturationMask)
  maskedimage = maskedimage.addBands(
    ee.Image([qa.bitwiseAnd(WaterBitMask).eq(0).rename('Water')])
    ).copyProperties(image).set('system:time_start', image.get('system:time_start'));
    
    
  return (maskedimage)


# Remove clouds, snow
landsat8FiltMasked = ls8.map(maskL8sr)


In [11]:
# Function to add NDVI, time, and constant variables to Landsat 8 imagery.
def addVariables(image):
  #Return the image with the added bands.
  return(image
         .addBands(image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))
         .addBands(image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')))
  
imgP = landsat8FiltMasked.map(addVariables)

# Select appropriate bands
imgO = imgP.select(['SR_B2', #blue
                    'SR_B3',# green
                    'SR_B4', # red
                    'SR_B5', # IR
                    'NDVI', 
                    'NDWI'])


In [12]:

####  set values for LS8 NDVI in range [0,1]

imgO = imgO.map(lambda img: set_values(img))


In [13]:
def func_apr(year):
    """Calculate annual mean for Bands"""
    date_start = ee.Date.fromYMD(year, 1, 1)
    date_end = date_start.advance(1, "year")

    image1 = (imgO.filterDate(date_start, date_end) 
        .reduce(ee.Reducer.mean(), TILES) 
        .set({"year": year}) 
        .set({"system:time_start": date_start.millis(), "system:time_end": date_end.millis()}))

    return(image1)



# for each year return mean composite of bands
annual_mean_NDVI8 = ee.ImageCollection.fromImages(
    ee.List.sequence(
        ee.Date(DATES_LS8[0]).get("year").getInfo(),
        ee.Date(DATES_LS8[1]).get("year").getInfo() - 1,
    ).map(func_apr)
)

# Merge the two timeseries

In [14]:
annual_mean_NDVI = annual_mean_NDVI5.merge(annual_mean_NDVI8)

# Generate a Water mask 

In [15]:
# Merge Water bands from two timeseries as a new imagecollection
water_col = landsat5FiltMasked.select("Water").merge(landsat8FiltMasked.select("Water"))

# reduce based MODE function and get one all time image composite  with TRUE/FALSE for water
water = water_col.select('Water').reduce(ee.Reducer.mode(),TILES)
def func_gtr(image):
  '''mask everything that is Water'''
  return image.updateMask(water.select("Water_mode").eq(1))

# mask timeseries with all time composite WATER mask
annual_mean_NDVI = annual_mean_NDVI.map(func_gtr)


# Map for an image

In [16]:
# Get the first image from the Full timeseries imagecollection
myyear = 2020
myimage = annual_mean_NDVI.filter(ee.Filter.eq("year", myyear)).first()

# Define visualization parameters
vis_params = {
    "bands": ["NDVI_mean"],  # Change bands if needed
    "min": -1,
    "max": 1, 
    "palette": ["red", "yellow", "green"]
}

# Create a map
Map = geemap.Map(center=[51.94, 16.35], zoom=5)  # Adjust center and zoom as needed

# Add the first image to the map
Map.addLayer(myimage, vis_params, f"LandSat 8 ({myyear})")
Map

Map(center=[51.94, 16.35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
# Export the image of year X to Google Drive
geemap.ee_export_image_to_drive(
    image=first_image.select('NDVI_mean'),
    description=f'NDVI_{myyear}.tif',
    folder='EXPORT_GEE',     # Google Drive folder name
    fileNamePrefix=f'NDVI_{myyear}',    # Output filename
    scale=100,
    crs='EPSG:3035',
    region=nuts.geometry().bounds(),
    fileFormat='GeoTIFF',
    maxPixels=1e13
)

# Prepare to export timeseries imagecollection 

In [17]:
# Select and rename 'NDVI_mean' band to NDVI_{year} band
def rename_band(image):
    year = ee.Number(image.get('year')).format('%d')  # Get the Year property
    new_band_name = ee.String('NDVI_').cat(year)
    
    image = image.select(['NDVI_mean']).rename(new_band_name)
    
    return image

NDVI_LS = annual_mean_NDVI.map(rename_band)

# Convert imagecollection to Multiband Image 
NDVI_LS = NDVI_LS.toBands()


# Get current band names
band_names = NDVI_LS.bandNames()

# Function to remove the prefix
def remove_prefix(band_name):
    return ee.String(band_name).split('_').slice(2).join('_')

# Apply the function to all band names
new_band_names = band_names.map(remove_prefix)

# Rename the bands in the image
NDVI_LS = NDVI_LS.rename(new_band_names)


# convert to float32
NDVI_LS = NDVI_LS.toFloat()


# Mean composite of all annual mean images

In [18]:
# based on this image percentiles per NUTS will be calculated

ndvi_mean_compo = NDVI_LS.reduce(ee.Reducer.mean())


# remove negatives
# Create a mask where the band values are greater than 0.
mask = ndvi_mean_compo.select('mean').gt(0)

# Update the image with the mask.
ndvi_mean_compo = ndvi_mean_compo.updateMask(mask)


# Export as Assets

In [20]:
# export NDVI_LS timeseries 
# Export as Asset
assetId = cfg_NDVI_LS_mean_ts_ASSET
geemap.ee_export_image_to_asset(
    NDVI_LS, description=assetId, assetId=assetId, region=nuts.geometry(),
    scale=100, 
    crs="EPSG:3035",
    maxPixels=1e13)

## Export composite Mean as Asset

First save it as Asset and the read it again in the next step (*NUTS percentiles and mean*). Else a memory issue occurs


In [21]:

assetId = cfg_NDVI_LS_mean_composite_ASSET
geemap.ee_export_image_to_asset(
    ndvi_mean_compo, 
    description=assetId, 
    assetId=assetId, 
    region=nuts.geometry(),
    scale=100, 
    crs="EPSG:3035",
    maxPixels=1e13)

# NUTS percentiles and mean

❗ SOS: run the following *cfg_NDVI_LS_mean_composite_ASSET* when is done in GEE

In [24]:
ndvi_mean_compo = ee.Image(f"{cfg_gee_assets}{cfg_NDVI_LS_mean_composite_ASSET}")


# use mean raster composite to calculate for each polygon the percentiles 2,95 and the mean

# Define mean and percentile reducers
mean_reducer = ee.Reducer.mean()
percentile_reducer = ee.Reducer.percentile([2, 95])

# Combine the reducers
combined_reducer = mean_reducer.combine(percentile_reducer, sharedInputs=True)

# apply reduceRegions
percentiles_NDVI = ndvi_mean_compo.reduceRegions(
    reducer = combined_reducer,
    collection = nuts,
    crs = "EPSG:3035",
    scale = 100,
    # bestEffort=True,
    # maxPixels = 1e13,
    tileScale = 16,
)


# export reduceRegions as a table for mean and percentiles per nuts 
task = ee.batch.Export.table.toDrive(
    collection= percentiles_NDVI,# ee.FeatureCollection([ee.Feature(None, Max_NDVI)]),
    folder = cfg_GDRIVE_EXPORT_DIRECTORY ,
    description=cfg_NDVI_LS_EL_mean_percentiles_2_95_mean_CSV,
    fileFormat='CSV',
    selectors= ['LEVL_CODE', 'NAME_LATN', 'NUTS_ID', 'NUTS_NAME', 'p2','p95', 'mean']
)
task.start()


# Export geotiffs to GDRIVE
Export σαν geotiff στο GDrive, για Θεσσαλία (EL61) και Αττική (EL30), mean NDVI του 2020

In [ ]:

# It processes and exports masked images from an Earth Engine Image 
# using specific geographic regions defined by NUTS_IDs.
myyear = 2020
img = annual_mean_NDVI.filter(ee.Filter.eq("year", myyear)).first().select('NDVI_mean')
NUTS_IDS =["EL61", "EL30"]

for NUTSID in NUTS_IDS:
    
    feat = nuts.filter(ee.Filter.eq("NUTS_ID", NUTSID))
    # create a mask based on current feature
    mask = ee.Image.constant(1).clip(feat)
        
    masked_img = img.updateMask(mask)

    export_task = ee.batch.Export.image.toDrive(
        image=masked_img,
        description=f'NDVI_LS_mean_composite_{NUTSID}_{myyear}',
        folder='EXPORT_GEE_NDVI_GEOTIFFS',  # Optional: Folder in Google Drive
        region=feat.geometry(),  # ROI in geo-coordinates
        scale=100,  # Scale in meters per pixel (adjust as needed)
        crs='EPSG:3035',  # Optional: Specify coordinate reference system
        maxPixels=1e13  # Maximum pixels (may need adjusting for large images)
    )
    export_task.start()